![alt text](header.png "MaxQuantSummerSchool2018")

# Machine Learning - Spectra Intensities Prediction

![alt text](alienprotein.png "Alien")

In [113]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,initializers
from keras.optimizers import Adagrad
from keras import metrics
import keras.backend as K
import matplotlib.pyplot as plt
import h5py
import keras
from keras.callbacks import ModelCheckpoint
import os
import tensorflow as tf
import pandas as pd
import csv
import re

In [114]:
'''
Created on Mar 10, 2017

@author: salinas
'''

from sklearn.svm import SVR
import numpy as np
import re
from sqlalchemy import create_engine
from pandas.io import sql
from sklearn.externals import joblib
from matplotlib import pyplot
import pandas as pd


class createWindowData(object):
    _daa=dict()
    _sequence=""
    _yIons=[]
    _bIons=[]
    _yIonsNorm = []
    _bIonsNorm = []
    _matrix=[]
    
    def __init__(self,sequence,ions,intensities):
        self._sequence=sequence
        #Creating Amino Matrix
        aa="-AEPRV"
        MAA=np.identity(len(aa))
        self._daa=dict()
        for index, arrayAA in enumerate(MAA):
            self._daa[aa[index]]=arrayAA                   
        #print  self._daa
    
        self.yIons=[0]*len(sequence)
        self.bIons=[0]*len(sequence)
        lstions=ions.split(";")
        lstintensities=intensities.split(";")
      
        yIonsreg = re.compile('^y[0-9]+$')
        bIonsreg = re.compile('^b[0-9]+$')

        self._yIons=[0]*len(sequence)
        self._bIons=[0]*len(sequence)
        
        for index, ion in enumerate(lstions):
            #print ion , lstintensities[index]
            if yIonsreg.match(ion):#                              
                self._yIons[int(ion.split("y")[1])] =float(lstintensities[index])            
            if bIonsreg.match(ion):
                self._bIons[int(ion.split("b")[1])] =float(lstintensities[index])
        
    
        list_b_ions=self._bIons[1:]
        list_b_ions.append(0)
        list_y_ions = self._yIons[::-1]
        self._bIons=list_b_ions
        self._yIons=list_y_ions

        self._matrix=[]
    
        
        
    def GenerateMatrix(self, size):
        dictFeature=dict()
        order=[] 
        size=int(size)
        for index in range(((size)//2)-1):
            key="Sj-"+str(index+1)        
            dictFeature[key]=["?"]*20
            order.append((index+1)*-1)
        ##############
        dictFeature["Sj"]=["?"]*20 
        ##############
        for index  in range((size)//2):
            key="Sj+"+str(index+1)
            dictFeature[key]=["?"]*20
            order.append(index+1)        
        ##########################
        dictFeature["Sj+1"]=["?"]*20
        dictFeature["Sj+2"]=["?"]*20
        ############################
        dictFeature["S1"]=["?"]*20
        dictFeature["SN"]=["?"]*20
        dictFeature["length"]="?"
        dictFeature["Dist-1"]="?"
        dictFeature["Dist-N"]="?"
        Sj_Positive= re.compile('^Sj\+')
        Sj_Negative = re.compile('^Sj-')
        order.append(0)
        order=sorted(order)
        #print self._sequence
        for index, aa in enumerate(self._sequence): 
            for k in  dictFeature:           
                val=0
                if Sj_Negative.match(k):
                    val=int(k[3:])*-1 
                   
                if Sj_Positive.match(k):
                    val=int(k[3:])  
                                         
                if (index+val)<0 or (index+val)>=len(self._sequence):
                    dictFeature[k]=self._daa["-"]
                else:
                    dictFeature[k]=self._daa[self._sequence[index+val]]

            #dictFeature["Sj"]
            dictFeature["Sj"]=self._daa[self._sequence[index]]
            #dictFeature["S1"]   
            dictFeature["S1"]=self._daa[self._sequence[0]]
            #dictFeature["SN"]
            dictFeature["SN"]=self._daa[self._sequence[len(self._sequence)-1]]
            #dictFeature["length"]
            dictFeature["length"]=len(self._sequence)
            #dictFeature["Dist-1"]="?"
            dictFeature["Dist-1"]=index
            #dictFeature["Dist-N"]="?"
            dictFeature["Dist-N"]=len(self._sequence)-index-1
            #print aa      
            #print dictFeature
            col=[]
            for i in order:
                if (i<0):
                    col+=list(dictFeature["Sj"+str(i)])
                if (i==0):
                    col+=list(dictFeature["Sj"]) 
                if (i>0):
                    col+=list(dictFeature["Sj+"+str(i)])
                    
            col+=list(dictFeature["S1"])+list(dictFeature["SN"])+[dictFeature["length"]]+[dictFeature["Dist-1"]+1]+[dictFeature["Dist-N"]]
            res= [int(i) for i in col ]
            #print dictFeature
            self._matrix.append(res)   
            
    def nomalizeIones(self):
        self._yIonsNorm = [float(i)/max(self._yIons+self._bIons) for i in self._yIons]
        self._bIonsNorm = [float(i)/max(self._yIons+self._bIons) for i in self._bIons]
    
    ### the zeros here are the ones with zero values in the intensities b and y ions 
    def GenerateDataset(self, removeZeros):
        self.nomalizeIones()    
        data={'featureMatrix':[],'target_Y':[], 'target_B':[] }
        
        for index, line in enumerate(self._matrix):
            if removeZeros==True:
                if (self._yIonsNorm[index]!=0 ):
                    data['featureMatrix'].append(line)
                    #data['target_Y'].append(("%.2f" %  self._yIonsNorm[index]))
                    data['target_Y'].append((self._yIonsNorm[index]))
 
                if (self._bIonsNorm[index]!=0):
                    #data['featureMatrix_B'].append(line)
                    data['target_B'].append((self._bIonsNorm[index]))
            else:
                data['featureMatrix'].append(line)
                data['target_Y'].append((self._yIonsNorm[index]))
                #data['featureMatrix_B'].append(line)
                data['target_B'].append((self._bIonsNorm[index]))   
        data['target_Y']=list(map(float,data['target_Y']))
        data['target_B']=list(map(float,data['target_B'])) 
        #Correction Bug
        data['featureMatrix']=data['featureMatrix'][:-1]
        data['target_Y']=data['target_Y'][:-1]
        data['target_B']=data['target_B'][:-1]
        return data        


In [115]:
#Generating the feature matrix
def input_data(sequence, ions, intensities,window_size):
    myTrainingMatrix=createWindowData(sequence,ions,intensities)
    myTrainingMatrix.GenerateMatrix(window_size)
    d=myTrainingMatrix.GenerateDataset(removeZeros=False)
    dm=d['featureMatrix']
    dmy=d['target_Y']
    dmb=d['target_B']
    list1 = range(0,len(dm[0]))
    list2 = ["C"] * len(dm[0])
    list3 = ["Yions","Bions"]
    feature_name = [b + str(a) for b,a in zip(list2,list1)]+list3
    arr_features=np.array(dm)
    arr_target=np.array([dmy,dmb])
    pd.set_option('display.max_columns', 500)
    test_data=pd.DataFrame(np.concatenate([arr_features,arr_target.T],axis=1),columns=feature_name)
    return test_data

Lets generate the feature space for the folowing peptide

In [119]:
#use only # A,E,P,R,V# 
sequence="AAAAV"
ions="y1;y2;b1"
intensities="20;40;20"
input_data(sequence, ions, intensities,2)
   

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,Yions,Bions
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,4.0,0.0,0.5
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,3.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,3.0,2.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,1.0,0.5,0.0


In [121]:
from sklearn.model_selection import train_test_split
aliendata = pd.read_csv("train_data.csv", header=None)
print(aliendata.shape)
train, val = train_test_split(aliendata, test_size=0.2)
X_train = train.iloc[:, :-2]
Y_train = train.iloc[:, 63:]
X_val=val.iloc[:, :-2]
Y_val=val.iloc[:, 63:]
print("training data size")
print(X_train.shape)
print("target size")
print(Y_train.shape)

(260, 65)
training data size
(208, 63)
target size
(208, 2)


Lets train our model with keras library 

In [137]:
NUM_LAYERS = 2
HIDDEN_UNITS = 32
OUTPUT_UNITS = 2
INPUT_DIM = 63
BATCH_SIZE = 8
DROPOUT_RATE = 0.0125
LEARNING_RATE = 0.01
NUM_EPOCHS = 5

In [138]:
class MetricsCallback(keras.callbacks.Callback):
    def __init__(self, metrics, validation_data):
        super().__init__()
        #self.validation_data = validation_data
        self.validation_data = validation_data
        self.metrics = metrics
model= Sequential()
for x in range(NUM_LAYERS): 
    model.add(Dense(HIDDEN_UNITS,kernel_initializer='random_normal',bias_initializer='zeros',activation='relu',input_dim=(INPUT_DIM if x == 0 else HIDDEN_UNITS)))
    model.add(Dropout(DROPOUT_RATE))
    model.add(Dense(OUTPUT_UNITS,kernel_initializer='random_normal',bias_initializer='zeros',activation='relu',input_dim=(HIDDEN_UNITS)))
adagrad = Adagrad(lr=LEARNING_RATE)
model.compile(loss='mean_squared_error',optimizer="adagrad", metrics=['mean_squared_error'])
metrics_callback = MetricsCallback(validation_data=(X_val, Y_val),
                                    metrics=['mean_squared_error'])
    
model.fit(X_train, Y_train,epochs=NUM_EPOCHS,batch_size=BATCH_SIZE,validation_data=(X_val, Y_val),callbacks=[metrics_callback])


Train on 208 samples, validate on 52 samples
Epoch 1/5
208/208 [==============================] - 2s 8ms/step - loss: 0.0804 - mean_squared_error: 0.0804 - val_loss: 0.0683 - val_mean_squared_error: 0.0683
Epoch 2/5
208/208 [==============================] - 0s 245us/step - loss: 0.0664 - mean_squared_error: 0.0664 - val_loss: 0.0682 - val_mean_squared_error: 0.0682
Epoch 3/5
208/208 [==============================] - 0s 327us/step - loss: 0.0621 - mean_squared_error: 0.0621 - val_loss: 0.0677 - val_mean_squared_error: 0.0677
Epoch 4/5
208/208 [==============================] - 0s 279us/step - loss: 0.0609 - mean_squared_error: 0.0609 - val_loss: 0.0660 - val_mean_squared_error: 0.0660
Epoch 5/5
208/208 [==============================] - 0s 250us/step - loss: 0.0578 - mean_squared_error: 0.0578 - val_loss: 0.0649 - val_mean_squared_error: 0.0649


Lets test our model now 

In [140]:
sequence="AAVPP"
ions="y1;b1"
intensities="1;1"
test_data = input_data(sequence, ions, intensities,8)
nw=test_data.iloc[:, :-2]
predictions = model.predict(nw)
rounded = [x for x in predictions]
rounded

[array([0.26681668, 0.09912269], dtype=float32),
 array([0.27094677, 0.10064127], dtype=float32),
 array([0.30867088, 0.11451198], dtype=float32),
 array([0.15466733, 0.05788665], dtype=float32)]

# LSTM Example

In [2]:
import os
import re
import pandas as pd
import collections
import sys
from numpy import array
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed, Reshape, Lambda
from keras import backend as K
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM 
import argparse
import numpy as np
from keras.utils import to_categorical

In [3]:
_RESIDUE = re.compile(r'[A-Z](\((\w+)\))?')
aa=['A','E','P','R','V']
peptides=["AAAAV","AAPR","AAE","ARV"]

data_training_y = [[100,0,0,50],
                   [60,0,30],
                  [10,10],
                   [10,10]]
pady = pad_sequences(data_training_y, maxlen=5, padding="post")
print(pady)
print(pady.shape)

one_hot_encoding = pd.get_dummies(aa).astype(int).to_dict(orient='list')
print (one_hot_encoding)
batch_peptidesss=[]
batch_peptides=[]
pad_list=[]
for peptide in peptides:
    input_features=[]
    for residue in re.finditer(_RESIDUE,peptide):
        input_list = one_hot_encoding[residue.group()]
        input_features.append(input_list)
    input_features = np.array(input_features,dtype=np.float64)
    batch_peptides.append(input_features)
    pad_list = pad_sequences(batch_peptides, maxlen=6, padding="post")
batch_peptidesss= np.array(pad_list, dtype=object)
print(batch_peptidesss.shape)

#### Model 
model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(None, 5))) # 5 becuase we have one hote conding to 5 [0,0,0,0,0]
model.add(Dense(5, activation='sigmoid')) # Five beacuse we have 5 Intensities to predict Y ions

# try using different optimizers and different optimizer configs
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
print('Train...')
model.fit(batch_peptidesss,pady,epochs=200)
pred = model.predict(batch_peptidesss)
print (pred)


[[100   0   0  50   0]
 [ 60   0  30   0   0]
 [ 10  10   0   0   0]
 [ 10  10   0   0   0]]
(4, 5)
{'P': [0, 0, 1, 0, 0], 'A': [1, 0, 0, 0, 0], 'V': [0, 0, 0, 0, 1], 'E': [0, 1, 0, 0, 0], 'R': [0, 0, 0, 1, 0]}
(4, 6, 5)
Train...
Epoch 1/200
4/4 [==============================] - 1s 199ms/step - loss: 856.3234 - acc: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 856.0046 - acc: 0.7500
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 855.8188 - acc: 1.0000
Epoch 4/200
4/4 [==============================] - 0s 2ms/step - loss: 855.5476 - acc: 1.0000
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 855.2787 - acc: 1.0000
Epoch 6/200
4/4 [==============================] - 0s 2ms/step - loss: 855.6745 - acc: 1.0000
Epoch 7/200
4/4 [==============================] - 0s 2ms/step - loss: 854.7135 - acc: 1.0000
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 854.7070 - acc: 1.0000
Epoch 9/200


Epoch 85/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8041 - acc: 1.0000
Epoch 86/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8065 - acc: 1.0000
Epoch 87/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8142 - acc: 1.0000
Epoch 88/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8037 - acc: 1.0000
Epoch 89/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8108 - acc: 1.0000
Epoch 90/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8079 - acc: 1.0000
Epoch 91/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8103 - acc: 1.0000
Epoch 92/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8037 - acc: 1.0000
Epoch 93/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8215 - acc: 1.0000
Epoch 94/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8035 - acc: 1.0000
Epoch 95/200
4/4 [==============================] 

Epoch 171/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8029 - acc: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8080 - acc: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8063 - acc: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8024 - acc: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8022 - acc: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8024 - acc: 1.0000
Epoch 177/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8022 - acc: 1.0000
Epoch 178/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8023 - acc: 1.0000
Epoch 179/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8025 - acc: 1.0000
Epoch 180/200
4/4 [==============================] - 0s 2ms/step - loss: 842.8039 - acc: 1.0000
Epoch 181/200
4/4 [=====================